<a href="https://colab.research.google.com/github/abs-git/RecSys/blob/main/Movie_RecSys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np

credits_path = '/content/gdrive/MyDrive/Colab Notebooks/RecSys/Movie/tmdb_5000_credits.csv'
movies_path = '/content/gdrive/MyDrive/Colab Notebooks/RecSys/Movie/tmdb_5000_movies.csv'


movies = pd.read_csv(credits_path)
credits = pd.read_csv(movies_path)



In [3]:
movies.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [5]:
credits.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [6]:
movies = movies.merge(credits, on = 'title')

In [7]:
movies.head(1)

,movie_id,title,cast,crew,budget,genres,homepage,id,keywords,original_language,...,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,vote_average,vote_count
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,...,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,7.2,11800


In [43]:
movies['original_language'].value_counts()[:5]

en    4510
fr      70
es      32
zh      27
de      27
Name: original_language, dtype: int64

In [9]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   movie_id              4809 non-null   int64  
 1   title                 4809 non-null   object 
 2   cast                  4809 non-null   object 
 3   crew                  4809 non-null   object 
 4   budget                4809 non-null   int64  
 5   genres                4809 non-null   object 
 6   homepage              1713 non-null   object 
 7   id                    4809 non-null   int64  
 8   keywords              4809 non-null   object 
 9   original_language     4809 non-null   object 
 10  original_title        4809 non-null   object 
 11  overview              4806 non-null   object 
 12  popularity            4809 non-null   float64
 13  production_companies  4809 non-null   object 
 14  production_countries  4809 non-null   object 
 15  release_date         

In [10]:
main_data = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [11]:
main_data.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [12]:
main_data.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [ ]:
main_data.dropna(inplace = True)

In [14]:
main_data.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [15]:
import ast

def genre_converter(obj):
  temp = []
  for i in ast.literal_eval(obj):
    temp.append(i['name'])
  return temp

def keyword_converter(obj):
  temp = []
  for i in ast.literal_eval(obj):
    temp.append(i['name'])
  return temp

def cast_converter(obj):
  temp = []
  counter = 0
  for i in ast.literal_eval(obj):
    if counter != 3:
      temp.append(i['name'])
      counter += 1
    else:
      break
  return temp

def fetch_director(obj):
  temp = []
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director':
      temp.append(i['name'])
      break
  return temp


In [ ]:
main_data['genres'] = main_data['genres'].apply(genre_converter)
main_data['keywords'] = main_data['keywords'].apply(keyword_converter)
main_data['cast'] = main_data['cast'].apply(cast_converter)
main_data['crew'] = main_data['crew'].apply(fetch_director)
main_data['overview'] = main_data['overview'].apply(lambda x : x.split())

In [17]:
main_data.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]


In [ ]:
# 띄어쓰기 제거

main_data['genres'] = main_data['genres'].apply(lambda x : [i.replace(" ", "") for i in x])
main_data['keywords'] = main_data['keywords'].apply(lambda x : [i.replace(" ", "") for i in x])
main_data['cast'] = main_data['cast'].apply(lambda x : [i.replace(" ", "") for i in x])
main_data['crew'] = main_data['crew'].apply(lambda x : [i.replace(" ", "") for i in x])

In [23]:
main_data.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]


In [ ]:
main_data['tags'] = main_data['overview'] + main_data['genres']+ main_data['keywords'] + main_data['cast'] + main_data['crew']

In [25]:
new_df = main_data[['movie_id', 'title', 'tags']]

In [26]:
new_df.head(2)

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."


In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x : " ".join(x))

In [28]:
new_df['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())

In [31]:
new_df.head(2)

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."


In [32]:
# 유사한 의미를 가진 단어를 기준으로 stemming을 수행한다. 

from nltk.stem.porter import PorterStemmer

def stem(text):
  ps = PorterStemmer()
  y = []

  for i in text.split():
    y.append(ps.stem(i))

  return " ".join(y)


In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [34]:
# tags에서 불용어 (stopword) 제거 후 벡터화한다.

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features= 5000, stop_words = 'english')

vectors = cv.fit_transform(new_df['tags']).toarray()


In [42]:
cv.get_feature_names()[:5]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['000', '007', '10', '100', '11']

In [36]:
vectors.shape

(4806, 5000)

In [37]:
new_df['tags'][0]

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

In [38]:
# tags를 기준으로 tags와 tags간 코사인 유사도 (cosine similarity)를 구한다.

from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(vectors)   # 4806 x 4806 

print(similarity[0])

[1.         0.08346223 0.0860309  ... 0.04499213 0.         0.        ]


In [39]:
def recommend(movie):
  movie_index = new_df[new_df['title'] == movie].index[0]

  distances = similarity[movie_index]

  movies_rank = sorted(list(enumerate(distances)), reverse = True, key = lambda x : x[1])[1:6]

  for i in movies_rank:
    print(new_df.iloc[i[0]].title)

In [40]:
recommend('Avatar')

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.


In [ ]:
import pickle
pickle.dump(new_df, open('movies.pkl','wb'))
pickle.dump(similarity, open('similarity.pkl', 'wb'))